In [36]:
import os

from os.path import  join, isdir
from plan import Plan
from action import Action
from utils import load_from_folder
from multiprocess import Pool
import random
from logging import exception
import re

random.seed(42)

In [99]:
save_dir = './new_plans/'
data_base_dir = '../datasets/'
domain = 'logistics'
results_dir = f"{save_dir}/{domain}/"   
source_dir = f"{join(data_base_dir, domain)}/optimal_plans/dictionaries_and_plans/" 
print('Domain dir:', source_dir)
os.makedirs(save_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

plans_to_process = 10 # number of plans to process
versions_per_plan = 5 # number of versions per each plan
number_of_goals = 2 # number of goals per each new plan
test = False # test will process only 3 plans

Domain dir: ../datasets/logistics/optimal_plans/dictionaries_and_plans/


In [22]:
plans = load_from_folder(source_dir,["plans"])[0]
print(f"Plans: {len(plans)}")

plans loaded from ../datasets/logistics/optimal_plans/dictionaries_and_plans/
Plans: 47769


In [ ]:
#process the objects in the plan then create the variations
count = 0
for plan in plans:
    
    if test:
        if count >= 3:
            break
    elif count > plans_to_process:
        break
    
    #begin plan processing
    
    #extract the name of the current plan with a regex
    name = re.search(r"(p\d+)(?=\.)", plan.plan_name).group(1)
    
    all_obj_set = set()
    package_for_goal_set = set()
    pos_for_goal_set = set()
    
    #* find all objects in the initial state and actions
    for line in plan.initial_state:
        for obj in line.split(" ")[1:]:
            all_obj_set.add(obj)
    for action in plan.actions:
        for fluent in action.positiveEffects:
            for obj in fluent.split(" ")[1:]:
                all_obj_set.add(obj)
        for fluent in action.negativeEffects:
            for obj in fluent.split(" ")[1:]:
                all_obj_set.add(obj)
        for fluent in action.precondition:
            for obj in fluent.split(" ")[1:]:
                all_obj_set.add(obj)
    
    #split the objects in their types
    pos_set = set()
    apn_set = set()
    cit_set = set()
    apt_set = set()
    tru_set = set()
    pack_set = set()
    for obj in all_obj_set:
        if obj.startswith("pos"):
            pos_set.add(obj)
            pos_for_goal_set.add(obj) #these will be used for goal creation
        elif obj.startswith("obj"):
            pack_set.add(obj)
            package_for_goal_set.add(obj) #these will be used for goal creation
        elif obj.startswith("apn"):
            apn_set.add(obj)
        elif obj.startswith("cit"):
            cit_set.add(obj)
        elif obj.startswith("tru"):
            tru_set.add(obj)
        elif obj.startswith("apt"):
            apt_set.add(obj)
            
    
    #raise an exception if number of goals > number of packages or positions
    if number_of_goals > len(package_for_goal_set):
        raise exception(f"Number of goals {number_of_goals} is greater than the number of objects {len(package_for_goal_set)}")

    if number_of_goals > len(pos_for_goal_set):
        raise exception(f"Number of goals {number_of_goals} is greater than the number of positions {len(pos_for_goal_set)}")

    #* now we start creating the new versions of the problem from the current one
    for i in range(0, versions_per_plan):
        #working copy of the sets as we have to prevent a package being in different positions
        package_for_goal_set_copy = package_for_goal_set.copy()
        
        #? can two packages be in the same position?
        #? for example |at obj1 pos1| e |at obj2 pos1|
        pos_for_goal_set_copy = pos_for_goal_set.copy()
        
        #create a set of goals of the type [at obj pos] 
        #by picking a random package and a random position from the sets
        #todo how to not have same goal_sets in two different versions? do we care about it?
        goal_set= set()    
        for k in range(0, number_of_goals):
            random_package = random.choice(list(package_for_goal_set_copy))
            #print(f"Random package: {random_package}")
            package_for_goal_set_copy.remove(random_package) # so we don't have conflicting goals
            random_pos = random.choice(list(pos_for_goal_set_copy))
            #print(f"Random position: {random_pos}")
            #? can two packages be in the same position?
            #? for example |at obj1 pos1| e |at obj2 pos1|
            #pos_for_goal_set_copy.remove(random_pos)
            goal_set.add(f"at {random_package} {random_pos}") 
        #print(f"Goal set: {goal_set}")
    
        #* now we build the new pddl file with these goals
        
        #definition
        new_problem = f"(define (problem {domain}_{name}_{i})\n(:domain {domain})\n(:objects\n\t"
        
        #objects
        if len(apn_set) > 0:
            for apn in apn_set:
                new_problem += f"{apn} "
            new_problem += f"- airplane\n\t"
        if len(cit_set) > 0:
            for cit in cit_set:
                new_problem += f"{cit} "
            new_problem += f"- city\n\t"
        if len(apt_set) > 0:
            for apt in apt_set:
                new_problem += f"{apt} "
            new_problem += f"- airport\n\t"
        if len(tru_set) > 0:
            for tru in tru_set:
                new_problem += f"{tru} "
            new_problem += f"- truck\n\t"
        if len (pack_set) > 0:
            for pack in pack_set:
                new_problem += f"{pack} "
            new_problem += f"- package\n\t"
        if len(pos_set) > 0:
            for pos in pos_set:
                new_problem += f"{pos} "
            new_problem += f"- location\n"
        new_problem += f")\n"
        
        #initial state
        new_problem += f"(:init\n"
        for fluent in plan.initial_state:
            new_problem += f"\t{fluent}\n"
        new_problem += f")\n"
        
        #goals
        new_problem += f"(:goal (and\n"
        for goal in goal_set:
            new_problem += f"\t{goal}\n"
        new_problem += f"))\n)"
        #print(new_problem + "\n\n")
        
        #save the new problem in a file
        #naming convention is {current plan name_version number.pddl}
        new_problem_dir = f"{results_dir}/{name}/"
        os.makedirs(new_problem_dir, exist_ok=True)
        new_problem_file = f"{new_problem_dir}/{name}_{i}.pddl"
        with open(new_problem_file, "w") as f:
            f.write(new_problem)

    count = count + 1

Random package: obj12
Random position: pos13
Random package: obj44
Random position: pos55
Random package: obj12
Random position: pos21
Random package: obj55
Random position: pos44
Random package: obj33
Random position: pos13
Random package: obj66
Random position: pos44
Random package: obj12
Random position: pos23
Random package: obj11
Random position: pos44
Random package: obj12
Random position: pos21
Random package: obj11
Random position: pos66
Random package: obj99
Random position: pos55
Random package: obj77
Random position: pos55
Random package: obj44
Random position: pos66
Random package: obj33
Random position: pos66
Random package: obj66
Random position: pos23
Random package: obj33
Random position: pos21
Random package: obj99
Random position: pos23
Random package: obj77
Random position: pos66
Random package: obj33
Random position: pos55
Random package: obj44
Random position: pos22
Random package: obj11
Random position: pos55
Random package: obj55
Random position: pos55
Random pac

In [ ]:
#todo we need to create plans starting from other plans and a set difficulty class, changing a number of goals from the original plan